### Моделирование конечных автоматов

[Реализация конечных автоматов на python](https://github.com/pytransitions/transitions)

Пример 1: Описание изменения агрегатного состояния

In [ ]:
!pip install transitions
import transitions

from transitions import Machine

# На этот объект будем вешать состояния
class Matter(object):
    pass

lump = Matter()

# Полный список состояний
states=['solid', 'liquid', 'gas', 'plasma']

# Добавляем таблицу переходов — из какое в какое состояние мы можем попасть
transitions = [
    { 'trigger': 'melt', 'source': 'solid', 'dest': 'liquid' },
    { 'trigger': 'evaporate', 'source': 'liquid', 'dest': 'gas' },
    { 'trigger': 'sublimate', 'source': 'solid', 'dest': 'gas' },
    { 'trigger': 'ionize', 'source': 'gas', 'dest': 'plasma' }
]

    # trigger – это некое действие, которое может привести к смене состояния объекта,
    # source – исходное состояние объекта,
    # dest – целевое состояние объекта.


# Инициализация машины
machine = Machine(lump, states=states, transitions=transitions, initial='liquid')

# Проверяем начальное состояние
lump.state

# И пробуем изменить состояние триггерами перехода
lump.evaporate()
lump.state

lump.trigger('ionize')
lump.state

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 1.5 MB/s eta 0:00:00


'plasma'

In [ ]:
machine = Machine(lump, states=states, transitions=transitions, initial='solid')
try:
    lump.melt()
    print(lump.state)

    lump.evaporate()
    print(lump.state)

    lump.ionize()
    print(lump.state)

except MachineError as error:
    print(error)

liquid
gas
plasma


Пример 2.1: Конечный автомат, управляющий работой стиральной машины

In [ ]:
class State:
    def __init__(self, name, is_final=False):
        self.name = name
        self.is_final = is_final
        self.transitions = {}

    def add_transition(self, action, state):
        self.transitions[action] = state


class WashingMachine:
    def __init__(self, start_state):
        self.current_state = start_state

    def perform_action(self, action):
        if action in self.current_state.transitions:
            self.current_state = self.current_state.transitions[action]
            print(f"Переход в состояние: {self.current_state.name}")
        else:
            print(f"Действие '{action}' невозможно в состоянии '{self.current_state.name}'.")


# Определяем состояния
waiting = State("Ожидание", is_final=False)
washing = State("Стирка", is_final=False)
rinsing = State("Полоскание", is_final=False)
drying = State("Сушка", is_final=False)
finished = State("Завершено", is_final=True)

# Определяем переходы
waiting.add_transition('старт', washing)
washing.add_transition('полоскать', rinsing)
rinsing.add_transition('сушить', drying)
drying.add_transition('завершить', finished)

# Создаем стиральную машину
washing_machine = WashingMachine(start_state=waiting)

# Моделируем работу стиральной машины
actions = ['старт', 'полоскать', 'сушить', 'завершить']

for action in actions:
    washing_machine.perform_action(action)

# Проверка на завершение работы
if washing_machine.current_state.is_final:
    print("Стиральная машина завершила работу.")


Переход в состояние: Стирка
Переход в состояние: Полоскание
Переход в состояние: Сушка
Переход в состояние: Завершено
Стиральная машина завершила работу.


Пример 2.2: конечный автомат, управляющий стиральной машиной.

Стиральная машина работает в трех режимах: залив, стирка, слив.

Машина начинает работать по нажатию кнопки «Пуск».
После этого происходит залив воды до тех пор, пока датчик *d1* уровня воды не подаст сигнал о заполнения бака стиральной машины. Затем происходит стирка.
Эта операция ограничивается с помощью таймера *t*.

Если таймер исправен, то по истечении определенного времени он выдает сигнал о завершении стирки и стиральная машина переходит в режим слива воды.

Если таймер неисправен, то стиральная машина переходит в состояние «*дефект*», т.е. в состояние ожидания, которое может быть прервано только после ремонта стиральной машины.

Из состояния «дефект» стиральная машина не может возвратиться в исходное состояние, поэтому последующее нажатие кнопки «Пуск» не приведет к запуску стиральной машины. Из неисправного состояния после ремонта стиральная машина возвращается в исходное состояние по сигналу *reset*.

Слив воды завершается при получении сигнала от датчика *d2* уровня воды о том, что в баке воды нет. После этого стиральная машина возвращается в исходное состояние.


In [ ]:
import time
import threading
from transitions import Machine

class WashingMachine:
    def __init__(self):
        # Определяем состояния
        self.states = [
            'initial',   # Исходное состояние
            'filling',   # Залив воды
            'washing',   # Стирка
            'draining',  # Слив воды
            'defect'     # Неисправное состояние
        ]

        # Определяем переходы
        self.transitions = [
            {'trigger': 'start', 'source': 'initial', 'dest': 'filling'},
            {'trigger': 'fill_complete', 'source': 'filling', 'dest': 'washing'},
            {'trigger': 'timer_complete', 'source': 'washing', 'dest': 'draining'},
            {'trigger': 'timer_failure', 'source': 'washing', 'dest': 'defect'},
            {'trigger': 'repair', 'source': 'defect', 'dest': 'initial'},
            {'trigger': 'drain_complete', 'source': 'draining', 'dest': 'initial'}
        ]

        # Создаем машину состояний
        self.machine = Machine(model=self, states=self.states, transitions=self.transitions, initial='initial')

        # Инициализируем состояния датчиков и таймер
        self.d1 = False  # Датчик для заполнения воды (True - вода заполнена)
        self.d2 = False  # Датчик для слива воды (True - вода слита)
        self.timer_running = False

    def fill_water(self):
        print("Заливаем воду...")
        # Имитация работы датчика d1
        time.sleep(2)  # Имитация времени заполнения воды
        self.d1 = True  # Вода заполнена
        print("Вода заполнена.")
        self.fill_complete()  # Переход к стирке

    def start_washing(self):
        print("Начинаем стирку...")
        self.timer_running = True
        timer_thread = threading.Thread(target=self.run_timer, args=(5,))  # Стирка 5 секунд
        timer_thread.start()

    def run_timer(self, duration):
        time.sleep(duration)
        if self.timer_running:
            print("Стирка завершена.")
            self.timer_complete()  # Переход к сливу
            self.drain_water()  # Автоматически запускаем слив после завершения стирки

    def drain_water(self):
        if self.state != 'draining':
            print("Ошибка: Машина не в состоянии слива!")
            return

        print("Сливаем воду...")
        # Имитация работы датчика d2
        time.sleep(2)  # Имитация времени слива воды
        self.d2 = True  # Вода слита
        print("Вода слита.")
        self.drain_complete()  # Завершение слива

    def wash(self):
        if self.state != 'washing':
            print("Ошибка: Машина не в состоянии стирки!")
            return

        if not self.d1:
            print("Ошибка: Вода не заполнена!")
            return

        self.start_washing()  # Начинаем стирку

    def timer_failure(self):
        print("Ошибка: Таймер не сработал.")
        self.timer_running = False  # Останавливаем таймер
        super().timer_failure()  # Вызываем метод перехода

    def reset_sensors(self):
        """Сброс датчиков при возврате в начальное состояние"""
        self.d1 = False
        self.d2 = False
        self.timer_running = False

# Пример использования
if __name__ == "__main__":
    wm = WashingMachine()

    print(f"Начальное состояние: {wm.state}")

    wm.start()  # Нажимаем кнопку "Пуск"
    print(f"Состояние после start: {wm.state}")

    wm.fill_water()  # Начинаем залив воды
    print(f"Состояние после fill_water: {wm.state}")

    wm.wash()  # Начинаем стирку
    # Дождаться завершения таймера
    time.sleep(6)
    print(f"Состояние после wash: {wm.state}")

    # Слив воды запускается автоматически из run_timer
    print(f"Состояние после drain_water: {wm.state}")

Начальное состояние: initial
Состояние после start: filling
Заливаем воду...
Вода заполнена.
Состояние после fill_water: washing
Начинаем стирку...
Стирка завершена.
Сливаем воду...
Состояние после wash: draining
Состояние после drain_water: draining
